In [53]:
import pickle
import numpy as np

import glob
import pandas as pd

fl = sorted(glob.glob('../features/*.pkl'))
for l in fl:
    print("'{}',".format(l))

'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_1_aug_1_fold8_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',
'../features/cnn_1_aug_skimage_denoise_feat.pkl',
'../features/cnn_1_aug_skimage_preprocess_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_2_aug_1_fold8_feat.pkl',
'../features/cnn_2_aug_denoise_preprocess_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_aug_skimage_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_3_aug_1_fold8_feat.pkl',
'../features/cnn_3_aug_denoise_preprocess_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug_skimage_preprocess_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_4_aug_1_fold8_feat.pkl',
'../features/cnn_4_aug_denoise_preprocess_feat.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../fea

In [54]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        #print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 56) (8424, 56) (1604,)


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]

        # train model
        m = LogisticRegression()
        m.fit(X_train,y_train)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict_proba(X_test))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        print('curr val loss', curr_val_loss)
        
        if test_pred is None:
            test_pred = m.predict_proba(test_x)
            w_test_pred = m.predict_proba(test_x)*curr_r_val_loss
        else:
            curr_pred = m.predict_proba(test_x)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    print(test_pred.shape)
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame(columns=['id','is_iceberg'])
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred[:,1]
        print(submission.head())
        submission.to_csv('../results/all_lr_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission['is_iceberg']=w_test_pred[:,1]
        print(submission.head())
        submission.to_csv('../results/weighted_lr_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

In [56]:
cv_test(3,42,True)

curr val loss 0.17250514
curr val loss 0.149592336871
curr val loss 0.167521660113
local average valid loss 0.163206378995 val loss std 0.00983921999488
(8424, 2)
         id  is_iceberg
0  5941774d    0.015250
1  4023181e    0.927541
2  b20200e4    0.013871
3  e7f018bb    0.986836
4  4371c8c3    0.751995
         id  is_iceberg
0  5941774d    0.015251
1  4023181e    0.927438
2  b20200e4    0.014212
3  e7f018bb    0.987066
4  4371c8c3    0.748066


In [57]:
cv_test(5,42,True)

curr val loss 0.185458156671
curr val loss 0.147440206936
curr val loss 0.172896028422
curr val loss 0.156557219055
curr val loss 0.167101765428
local average valid loss 0.165890675302 val loss std 0.0131222042904
(8424, 2)
         id  is_iceberg
0  5941774d    0.015231
1  4023181e    0.933889
2  b20200e4    0.008210
3  e7f018bb    0.987327
4  4371c8c3    0.753284
         id  is_iceberg
0  5941774d    0.015128
1  4023181e    0.934645
2  b20200e4    0.008315
3  e7f018bb    0.987441
4  4371c8c3    0.754875


In [58]:
cv_test(10,42,True)

curr val loss 0.185523119869
curr val loss 0.178969519124
curr val loss 0.173288988705
curr val loss 0.119992681865
curr val loss 0.1947546428
curr val loss 0.138849813311
curr val loss 0.164255325512
curr val loss 0.151445730826
curr val loss 0.165693066272
curr val loss 0.175401543369
local average valid loss 0.164817443165 val loss std 0.0213833928399
(8424, 2)
         id  is_iceberg
0  5941774d    0.014748
1  4023181e    0.935957
2  b20200e4    0.006748
3  e7f018bb    0.987536
4  4371c8c3    0.757940
         id  is_iceberg
0  5941774d    0.014599
1  4023181e    0.936214
2  b20200e4    0.006698
3  e7f018bb    0.987591
4  4371c8c3    0.758685
